## Проект для "Викишоп"

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В моём распоряжении набор данных с разметкой о токсичности правок. Задача: **построить модель со значением метрики качества F1 не меньше 0.75**. 

## Импорт, функции, константы

In [1]:
# Испортируем необходимые библиотеки

from tqdm import notebook
from tqdm.notebook import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import math

import re
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install spacy
import spacy

#!pip install transformers
import transformers
from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pip list | grep scikit

scikit-learn                      0.24.1
Note: you may need to restart the kernel to use updated packages.


## Знакомство с данными

In [2]:
#Распакуем наши данные
try:
    data = pd.read_csv('toxic_comments.csv', index_col = 0)
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)
display(data.head(3))
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [3]:
# Проверим дисбаланс классов
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Вывод

Для обучения модели нам был предоставлен DF с 159.292 наблюдениями. NaN значений нет, столбцы названы коррестно, единственная проблема - дисбаланс, её пофиксим на этапе прогноза или подготовки признаков.

## Подготовка признаков

In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sentence = "The striped bats are hanging on their feet for best"

doc = nlp(sentence)

" ".join([token.lemma_ for token in doc])


'the stripe bat be hang on their foot for good'

In [5]:
data['text'] = data['text'].values
data['text'] = data['text'].str.lower()

l = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_text(phrase):
    phrase_l = l(phrase)
    lemma_phrase = " ".join([token.lemma_ for token in phrase_l])
    finished_text = re.sub(r'[^a-zA-Z]', ' ', lemma_phrase) 
    return " ".join(finished_text.split())

data['lemma_text'] = data['text'].progress_apply(lemmatize_text)

data.head(10)

  0%|          | 0/159292 [00:00<?, ?it/s]

,text,toxic,lemma_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour I be see...
2,"hey man, i'm really not trying to edit war. it...",0,hey man I be really not try to edit war it be ...
3,"""\nmore\ni can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...
5,"""\n\ncongratulations from me as well, use the ...",0,congratulation from I as well use the tool wel...
6,cocksucker before you piss around on my work,1,cocksucker before you piss around on my work
7,your vandalism to the matt shirvington article...,0,your vandalism to the matt shirvington article...
8,sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense be offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which be contrar...


In [6]:
features = data['lemma_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

features_test, features_valid, target_test, target_valid = train_test_split(features_test, 
                                                                              target_test, 
                                                                              test_size=0.5, 
                                                                              random_state=12345)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train.values)
features_valid = count_tf_idf.transform(features_valid.values)
features_test = count_tf_idf.transform(features_test.values)

print(f'features_train shape is {features_train.shape}')
print(f'features_valid shape is {features_valid.shape}')
print(f'features_test shape is {features_test.shape}')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


features_train shape is (127433, 133938)
features_valid shape is (15930, 133938)
features_test shape is (15929, 133938)


## Обучение модели

In [7]:
# Логистическая регрессия

model_lr =LogisticRegression(C=10, solver='lbfgs', max_iter=10000, class_weight=1).fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
f1_score_lr = f1_score(target_valid, predictions_lr)
print(f'F1 метрика на valid данных: {f1_score_lr}')

F1 метрика на valid данных: 0.7909527073337903


In [8]:
# Дерево решений
model_dtc = DecisionTreeClassifier(max_depth=25).fit(features_train, target_train)
predictions = model_dtc.predict(features_valid)
f1_score_dct = f1_score(target_valid, predictions)
print(f'F1 метрика на valid данных: {f1_score_dct}')

F1 метрика на valid данных: 0.680029695619896


In [9]:
# Случайный лес

model_rfc = RandomForestClassifier(n_estimators = 60, max_depth=50).fit(features_train, target_train)
predictions = model_rfc.predict(features_valid)
f1_score_rfc = f1_score(target_valid, predictions)
print(f'F1 метрика на valid данных: {f1_score_rfc}')

F1 метрика на valid данных: 0.05162064825930372


In [ ]:
#CatBoost

model_cb = CatBoostClassifier(verbose=False, iterations=250).fit(features_train, target_train)
predictions = model_cb.predict(features_valid)
f1_score_cb_cv = cross_val_score(model_cb,
                                         features_train, 
                                         target_train, 
                                         cv=3, 
                                         scoring='f1').mean()
f1_score_cb = f1_score(target_valid, predictions)
print('F1 на кросс-валидации', f1_score_cb_cv)
print(f'F1 метрика на valid данных: {f1_score_cb}')

In [ ]:
f1_scores = pd.DataFrame(data = [round(f1_score_lr, 2), round(f1_score_dct, 2), round(f1_score_rfc, 2), round(f1_score_cb, 2)],
                         index=['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier'],
                         columns = ['F1'])
f1_scores = f1_scores.sort_values(by='F1', ascending=False)
display(f1_scores)

plt.figure(figsize=(12, 7))
sns.barplot(x = f1_scores.index, y = f1_scores['F1'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
predictions_lr = model_lr.predict(features_test)
f1_score_lr = f1_score(target_test, predictions_lr)
print(f'F1 метрика на valid данных: {f1_score_lr}')

In [ ]:
# Создадим матрицу ошибок случайного леса
cm_lr = confusion_matrix(target_test, predictions_lr)
cm_lr_data = pd.DataFrame(cm_lr,
                           index=['True Not Canceled', 'True Canceled'],
                           columns=['False Not Canceled', 'False Canceled'])

plt.figure(figsize=(15,8))
sns.heatmap(cm_lr, xticklabels=cm_lr_data.columns, yticklabels=cm_lr_data.index, annot=True, fmt='g', cmap="Reds", annot_kws={"size": 20})
plt.title("Матрица ошибок", size=20)
plt.xlabel('Предсказания', size=20)
plt.ylabel('Реальность', size=20);

## Вывод

Нам удалось обучить 4 модели на подготовленной выборке и проверить их на 20% от всех данных. Самый лучший результат показала логистическая регрессия - F1 мера приблизительно равна 78%, что является вполне приемлемым результатом, который позволит снизить работу администратора системы примерно в 5 раз.

Судя по матрице ошибок наша модель научилась отлично предсказывать нулевые классы - не токсичные комментарии, что не скажешь об обратных. Чтобы улучшим метрику и саму модель необходимо дать на вход намного больше отрицательных комментариев, возможно, сгенерировав их саморучно или с помощью чего-либо.